In [4]:
import torch
import torch.nn.functional as F
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from matplotlib import pyplot as plt


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.random.manual_seed(42)

In [8]:
# 定义模型
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(1, 10)
        self.fc2 = nn.Linear(10, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# 定义训练函数
def train(model, optimizer, criterion, train_loader, epochs, *, closure: bool = False):
    losses = []
    for epoch in range(epochs):
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            def step() -> torch.Tensor:
                optimizer.zero_grad()
                output = model(x)
                loss = criterion(output, y)
                loss.backward()
                return loss
            if closure:
                loss = optimizer.step(step)
            else:
                loss = step()
                optimizer.step()
            losses.append(loss.item())
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, epochs, loss.item()))
    return losses

# 定义数据集
class SinDataset(Dataset):
    def __init__(self, size=1000):
        self.x = torch.linspace(-2*torch.pi, 2*torch.pi, size).unsqueeze(1)
        self.y = torch.sin(self.x)

    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return len(self.x)

# 定义超参数
batch_size = 64
learning_rate = 0.003
epochs = 100

# 定义数据加载器
train_dataset = SinDataset()
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# 使用SGD优化器
model = Net().to(device)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
sgd_losses = train(model, optimizer, criterion, train_loader, epochs)

# 使用LBFGS优化器
model = Net().to(device)
criterion = nn.MSELoss()
optimizer = optim.LBFGS(model.parameters(), lr=learning_rate)
lbfgs_losses = train(model, optimizer, criterion, train_loader, epochs, closure=True)


# 绘制损失值变化图
plt.plot(sgd_losses, label='SGD')
plt.plot(lbfgs_losses, label='LBFGS')
plt.legend()
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.show()

Epoch [1/50], Loss: 0.5207
Epoch [2/50], Loss: 0.3070
Epoch [3/50], Loss: 0.1890
Epoch [4/50], Loss: 0.2461
Epoch [5/50], Loss: 0.2801
Epoch [6/50], Loss: 0.5081
Epoch [7/50], Loss: 0.2486
Epoch [8/50], Loss: 0.2729
Epoch [9/50], Loss: 0.2588
Epoch [10/50], Loss: 0.2928
Epoch [11/50], Loss: 0.2587
Epoch [12/50], Loss: 0.3112
Epoch [13/50], Loss: 0.3759
Epoch [14/50], Loss: 0.1688
Epoch [15/50], Loss: 0.2236
Epoch [16/50], Loss: 0.2042
Epoch [17/50], Loss: 0.2559
Epoch [18/50], Loss: 0.1795
Epoch [19/50], Loss: 0.2110
Epoch [20/50], Loss: 0.3016
Epoch [21/50], Loss: 0.1749
Epoch [22/50], Loss: 0.2471
Epoch [23/50], Loss: 0.2199
Epoch [24/50], Loss: 0.3038
Epoch [25/50], Loss: 0.1147
Epoch [26/50], Loss: 0.1895
Epoch [27/50], Loss: 0.1605
Epoch [28/50], Loss: 0.2326
Epoch [29/50], Loss: 0.1951
Epoch [30/50], Loss: 0.2067
Epoch [31/50], Loss: 0.1382
Epoch [32/50], Loss: 0.1731
Epoch [33/50], Loss: 0.1198
Epoch [34/50], Loss: 0.1992
Epoch [35/50], Loss: 0.2463
Epoch [36/50], Loss: 0.1696
E

: 

: 